In [1]:
import zipfile
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from keras.preprocessing import image
from random import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv
/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip


In [3]:
PATH = '/kaggle/input/dogs-vs-cats-redux-kernels-edition/'
TRAIN_DIR = './train/'
TEST_DIR = './test/'
img_width = 150
img_height = 150

In [4]:
train_image_path = os.path.join(PATH, "train.zip")
test_image_path = os.path.join(PATH, "test.zip")

**Unzip the train and test files.**

In [5]:
with zipfile.ZipFile(train_image_path, "r") as z:
    z.extractall(".")
    
with zipfile.ZipFile(test_image_path, "r") as z:
    z.extractall(".") 

In [6]:
train_images_list = [TRAIN_DIR + i for i in os.listdir(TRAIN_DIR)] 
test_images_list = [TEST_DIR + i for i in os.listdir(TEST_DIR)]

**We select 7500 cats and 7500 dogs images.**

In [7]:
cats_imgs = []
dogs_imgs = []


def img_seperator(img_list):
    for img in train_images_list:
        if 'cat' in img:
            cats_imgs.append(img)
        elif 'dog' in img:
            dogs_imgs.append(img)
    return cats_imgs, dogs_imgs

img_seperator(test_images_list)


cats_imgs = cats_imgs[:7500]
dogs_imgs = dogs_imgs[:7500]

train_images_list = cats_imgs + dogs_imgs

print(len(train_images_list))

15000


In [8]:
def process_data(list_of_images):
    x = []  # images as arrays
    y = []  # labels
    
    for img in list_of_images:
        img = cv2.imread(img)
        img = cv2.resize(img, (img_width,img_height))    
        x.append(img)
    
    for i in list_of_images:
        if 'dog' in i:
            y.append(1)
        elif 'cat' in i:
            y.append(0)
            
    return x, y      

In [9]:
X, y = process_data(train_images_list)

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

**CNN InceptionV3 model!**

In [11]:
local_weights_file = "../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [12]:
# Initialize the base model.
# Set the input shape and remove the dense layers.
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

# Load the pre-trained weights you downloaded.
pre_trained_model.load_weights(local_weights_file)

# Freeze the weights of the layers.
for layer in pre_trained_model.layers:
    layer.trainable = False

"""You can see the summary of the model below. You can see that it is a very deep network. You can then select up to which point of the network you want to use. As Laurence showed in the exercise, you will use up to `mixed_7` as your base model and add to that. This is because the original last layer might be too specialized in what it has learned so it might not translate well into your application. `mixed_7` on the other hand will be more generalized and you can start with that for your application. After the exercise, feel free to modify and use other layers to see what the results you get."""

#pre_trained_model.summary()

# Choose `mixed_7` as the last layer of your base model
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

2022-08-01 19:32:17.816107: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 19:32:17.939792: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 19:32:17.940744: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-01 19:32:17.943065: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

last layer output shape:  (None, 7, 7, 768)


**Add dense layers to the classifier**

**Next, we will add dense layers to your model. These will be the layers that we will train and is tasked with recognizing cats and dogs. We will add a Dropout layer as well to regularize the output and avoid overfitting.**

In [13]:
from tensorflow.keras import Model

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.5)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)           

# Append the dense network to the base model
model = Model(pre_trained_model.input, x) 

# Print the model summary. See your dense network connected at the end.
model.summary()

# Set the training parameters
model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [14]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.95):
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True

In [15]:
# All images will be rescaled by 1./255 and we also rotate and do other operations

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


val_datagen = ImageDataGenerator(rescale=1./255)


In [16]:
# Flow training images in batches of 20 using train_datagen generator

train_generator = train_datagen.flow(
        np.array(X_train),
        y_train,
        batch_size=20)

validation_generator = val_datagen.flow(
        np.array(X_val),
        y_val,
        batch_size=20)

In [17]:
# Train your model

callbacks = myCallback()
history = model.fit(train_generator,
                    steps_per_epoch=100,
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps=50,
                    callbacks=callbacks
                   )

2022-08-01 19:32:25.914774: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-08-01 19:32:29.517093: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


100/100 [==============================] - 21s 120ms/step - loss: 0.3665 - accuracy: 0.8440 - val_loss: 0.1239 - val_accuracy: 0.9520
Epoch 2/10
100/100 [==============================] - 12s 115ms/step - loss: 0.2578 - accuracy: 0.8970 - val_loss: 0.1926 - val_accuracy: 0.9310
Epoch 3/10
100/100 [==============================] - 11s 111ms/step - loss: 0.2695 - accuracy: 0.9060 - val_loss: 0.1598 - val_accuracy: 0.9420
Epoch 4/10
100/100 [==============================] - 11s 114ms/step - loss: 0.2642 - accuracy: 0.8985 - val_loss: 0.1015 - val_accuracy: 0.9630
Epoch 5/10
100/100 [==============================] - 11s 111ms/step - loss: 0.2548 - accuracy: 0.9090 - val_loss: 0.1484 - val_accuracy: 0.9520
Epoch 6/10
100/100 [==============================] - 12s 116ms/step - loss: 0.3128 - accuracy: 0.8965 - val_loss: 0.1491 - val_accuracy: 0.9450
Epoch 7/10
100/100 [==============================] - 11s 108ms/step - loss: 0.2610 - accuracy: 0.9105 - val_loss: 0.0950 - val_accuracy: 0.9

In [18]:
model.save_weights('model_weights.h5')
model.save('model_keras.h5')

**After the training and the validating steps, we are ready to make the test set.**

In [19]:
X_test, y_test = process_data(test_images_list) #Y_test in this case will be []

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [20]:
test_generator = val_datagen.flow(np.array(X_test), batch_size=20)

prediction_probabilities = model.predict_generator(test_generator, verbose=1)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


625/625 [==============================] - 10s 15ms/step


**Ready for Submission**

In [21]:
counter = range(1, len(test_images_list) + 1)
solution = pd.DataFrame({"id": counter, "label":list(prediction_probabilities)})
cols = ['label']

for col in cols:
    solution[col] = solution[col].map(lambda x: str(x).lstrip('[').rstrip(']')).astype(float)

solution.to_csv("submission.csv", index = False)